In [1]:
import osmium
import sys
from neo4j import GraphDatabase, basic_auth, unit_of_work
from copy import deepcopy

In [2]:
url = 'http://download.geofabrik.de/europe/monaco-latest.osm.pbf'

In [3]:
driver = GraphDatabase.driver('bolt://54.167.190.221:7687',
                              auth=basic_auth('neo4j', 'release-shows-breads'))

In [4]:
class GraphNode:
    def __init__(self, node_id, lat, long):
        self.node_id = node_id
        self.lat = lat
        self.long = long

    def __repr__(self):
        return f'{{node_id: {self.node_id}, lat: {self.lat}, long: {self.long}}}'

In [37]:
class GraphEdge:
    def __init__(self, start_node_id, end_node_id, distance, rating):
        self.start_node_id = start_node_id
        self.end_node_id = end_node_id
        self.distance = distance
        self.rating = rating

    def __repr__(self):
        return f'{{start_node_id: {self.start_node_id}, end_node_id: {self.end_node_id}, distance: {self.distance}, rating: {self.rating}}}'

In [55]:
class FileHandler(osmium.SimpleHandler):
    def __init__(self, neoHandler):
        super(FileHandler, self).__init__()
        self.neoHandler = neoHandler

    def node(self, n):
        id = deepcopy(n.id)
        lat, long = deepcopy(str(n.location)).split('/')
        node = GraphNode(n.id, lat, long)
        self.neoHandler.add_node(node)

    def way(self, w):
        if w.nodes[0].ref == w.nodes[-1].ref:
            return
        for i in range(len(w.nodes)-1):
            distance = osmium.geom.haversine_distance(w.nodes[i:i+1])
            edge = GraphEdge(w.nodes[i].ref, w.nodes[i+1].ref, distance, 4)
            self.neoHandler.add_edge(edge)

In [30]:
def batch_nodes(session, nodes, batch_size=4000):
    batch = []
    print('start nodes')
    for i, node in enumerate(nodes):
        if (i % batch_size == 0 and len(batch) > 0) or i == len(nodes) - 1:
            with session.begin_transaction() as tx:
                tx.run(
                    f'''FOREACH (node IN [{','.join(map(lambda x: str(x), batch))}] |
                       CREATE (:Node{{node_id:node.node_id,lat:node.lat,long:node.long}}))'''
                )
                tx.commit()
                print(f'Close transaction {i//batch_size}')
                batch = []
                continue
        batch.append(node)

In [49]:
def batch_edges(session, edges, batch_size=4000):
    batch = []
    print('start relations')
    for i, edge in enumerate(edges):
        if (i % batch_size == 0 and len(batch) > 0) or i == len(edges) - 1:
            print('begining transaction')
            with session.begin_transaction() as tx:
                tx.run(
                    f'''FOREACH (edge IN [{','.join(map(lambda x: str(x), batch))}] |
                    MATCH (a: Node),(b: Node)
                    WHERE a.node_id=edge.start_node_id AND b.node_id=edge.end_node_id
                    CREATE (a)-[r:Route {{
                    distance:edge.distance,
                    rating:edge.rating
                    }}])->(b)''')
                tx.commit()
                print(f'Close transaction {i//batch_size}')
                batch = []
                continue

        batch.append(edge)

In [45]:
class Neo4jHandler():
    def __init__(self):
        self.driver = driver
        self.nodes = []
        self.edges = []

    def add_node(self, node):
        self.nodes.append(node)

    def add_edge(self, edge):
        self.edges.append(edge)

    def create_objects(self):
        with self.driver.session() as session:
            batch_nodes(session, self.nodes)
            batch_edges(session, self.edges)

    def close(self):
        self.driver.close()

In [56]:
with driver.session() as session:
    session.run('MATCH (n) DETACH DELETE n')

In [57]:
neoHandler = Neo4jHandler()
h = FileHandler(neoHandler)

h.apply_file(url, locations=True)
print(f'Nodes: {len(neoHandler.nodes)}')

neoHandler.create_objects()
neoHandler.close()
print('finished')

TypeError: __getitem__(): incompatible function arguments. The following argument types are supported:
    1. (self: osmium.osm._osm.NodeRefList, arg0: int) -> osmium.osm._osm.NodeRef

Invoked with: osmium.osm.WayNodeList([osmium.osm.NodeRef(ref=21912089, location=osmium.osm.Location(x=74259518, y=437389494)), osmium.osm.NodeRef(ref=7265761724, location=osmium.osm.Location(x=74258602, y=437389997)), osmium.osm.NodeRef(ref=1079750744, location=osmium.osm.Location(x=74257964, y=437390370)), osmium.osm.NodeRef(ref=2104793864, location=osmium.osm.Location(x=74257430, y=437390849)), osmium.osm.NodeRef(ref=6340961560, location=osmium.osm.Location(x=74257083, y=437391243)), osmium.osm.NodeRef(ref=1110560507, location=osmium.osm.Location(x=74256818, y=437391636)), osmium.osm.NodeRef(ref=21912093, location=osmium.osm.Location(x=74256611, y=437392238)), osmium.osm.NodeRef(ref=6340961559, location=osmium.osm.Location(x=74256536, y=437392768)), osmium.osm.NodeRef(ref=21912095, location=osmium.osm.

nodes = []
with driver.session() as session:
    result = session.run('MATCH (n:Node) return n')
    for record in result:
        nodes.append([record[0].get('lat'), record[0].get('long')])

import numpy as np


def closest_node(node, nodes):
    nodes = np.asarray(nodes)
    deltas = nodes - node
    dist_2 = np.einsum('ij,ij->i', deltas, deltas)
    return np.argmin(dist_2)

arr = np.array([[10, 2], [12, -2], [3, 8]])
err = (arr - np.array([12, 4]))**2
delta = np.sum(err, axis=1)
pos = np.argmin(delta)
print(err, delta, pos)